In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

pd.set_option("display.max_columns", 85)

# Random Forest Evaluation with Nested Cross-Validation

## Load Data

In [ ]:
# Read Data from CSV (NEW DATA, NOT SCALED)
df = pd.read_csv("../data/abnormal_writeout_noscale.data.csv", index_col=0)

# trascurare da ACC a UVM
start_drop = df.columns.get_loc("ACC")
end_drop = df.columns.get_loc("UVM")
cols = np.arange(start_drop, end_drop + 1)
df.drop(df.columns[cols], axis=1, inplace=True)

# trascurare alcune colonne
df.drop("TTT_freq", axis=1, inplace=True)
df.drop("oldest_phylostratum_factor", axis=1, inplace=True)

# Drop NaNs
df.dropna(inplace=True)

# Sort features
resp = df["response"]
occ = df["occ_total_sum"]
age = df["oldest_phylostratum"]
conf = df.drop(labels=["response", "occ_total_sum", "oldest_phylostratum"], axis=1)

# Collect Features and Labels
features_df = pd.DataFrame()
features_df["occ_total_sum"] = occ
features_df["oldest_phylostratum"] = age
features_df = pd.concat([features_df, conf], axis=1)

X = features_df.to_numpy()
y = df["response"].to_numpy()

features_df.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

***
## Custom PCA

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Columns of confounder variables (highly colinear)
conf_index = 2
conf_cols = np.arange(2, X.shape[1])


class ConfounderPCA(BaseEstimator, TransformerMixin):
    """ 
    Custom PCA transformer for this dataset.
    Applies PCA only to the many collinear confounder 
    variables.
    
    cols - columns to which PCA will be applied.
    
    n_components - same as with the "vanilla" PCA. 
        If 0 < n_components < 1, select the number of 
        components such that the amount of variance that 
        needs to be explained is greater than the 
        percentage specified by n_components.
        
    apply_PCA - if false, simply returns the untransformed data.
    """

    def __init__(self, cols, n_components=None, apply_PCA=True):
        self.n_components = n_components
        self.apply_PCA = apply_PCA
        self.cols = cols
        if self.apply_PCA:
            self.pca = PCA(n_components=self.n_components)

    def fit(self, X, y=None):
        if self.apply_PCA:
            self.pca.fit(X[:, self.cols])
        return self

    def transform(self, X, y=None):
        if self.apply_PCA:
            X_pca = self.pca.transform(X[:, self.cols])
            return np.c_[X[:, :2], X_pca]
        else:
            return X


sns.heatmap(pd.DataFrame(ConfounderPCA(cols=np.arange(2, X.shape[1])).fit_transform(StandardScaler().fit_transform(X))).corr())
plt.title("Correlation Matrix after PCA")
plt.show()

print(X.shape[1], "total features.")
print("Confounder columns start from index", conf_index, "of feature matrix.")
print("Non-counfounders:", features_df.iloc[:, 0:conf_index].columns.tolist())

features_df

## The Model and its Parameter Space

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from imblearn.ensemble import BalancedRandomForestClassifier

param_grid = {
    'rf__max_depth': [None, 10, 20, 30],
    "rf__min_samples_leaf": [1, 2, 5, 10, 20,],
    "rf__min_samples_split": [2, 5, 10, 20],
    "rf__max_features": [None, "sqrt"],
    "rf__n_estimators": [100, 300, 1000],
    "pca__apply_PCA": [False, True],
    "pca__n_components": [0.01, 0.5, 0.95, None],
}
rf_clf = Pipeline([
    ('scaler', StandardScaler()),
    ("pca", ConfounderPCA(cols=np.arange(2, X.shape[1]))), 
    ("rf", BalancedRandomForestClassifier(random_state=42, n_jobs=-1))
])

## Nested CV

In [ ]:
from sklearn.exceptions import ConvergenceWarning, FitFailedWarning
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils._testing import ignore_warnings


# configure the cross-validation procedure
k_outer = 5
k_inner = 3
cv_outer = KFold(n_splits=k_outer, shuffle=True, random_state=1)

# To store results
roc_results = list()
# auprc_results = list()
# prec_results = list()
# rec_results = list()
# f1_results = list()
found_params = list()

print("OUTER CV | INNER CV | CHOSEN PARAMS")

for i, (train_ix, test_ix) in enumerate(cv_outer.split(X)):

    # split data
    X_train, X_test = X[train_ix, :], X[test_ix, :]
    y_train, y_test = y[train_ix], y[test_ix]

    # configure the cross-validation procedure
    cv_inner = KFold(n_splits=k_inner, shuffle=True, random_state=i) # Deterministic but changing RS

    with ignore_warnings(category=[ConvergenceWarning, FitFailedWarning]):
        # define search
        search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, scoring="roc_auc", cv=cv_inner)
        # execute search
        result = search.fit(X_train, y_train)
        
    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_

    # evaluate model on the hold out dataset
    # yhat = best_model.predict(X_test)
    yhat = best_model.predict_proba(X_test)[:,1]

    # evaluate the model
    roc_auc = roc_auc_score(y_test, yhat)
    
    # store the result
    roc_results.append(roc_auc)
    # auprc_results.append(auprc(y_test, yhat))
    # prec_results.append(precision_score(y_test, yhat))11
    # rec_results.append(recall_score(y_test, yhat))
    # f1_results.append(f1_score(y_test, yhat))
    found_params.append(result.best_params_)

    # report progress
    print("roc-auc=%.3f, est=%.3f, params=%s" % (roc_auc, result.best_score_, result.best_params_))

# summarize the estimated performance of the model
print("ROC-AUC: %.3f (std = %.3f)" % (np.mean(roc_results), np.std(roc_results)))
print("(other scores stored in ncv_df)")

In [ ]:
ncv_df = pd.DataFrame()
ncv_df["ROC-AUC"] = roc_results
# ncv_df["AUPRC"] = auprc_results
# ncv_df["Precision"] = prec_results
# ncv_df["Recall"] = rec_results
# ncv_df["f1-score"] = f1_results
ncv_df = pd.concat([ncv_df, pd.DataFrame(found_params)], axis=1)
ncv_df.to_csv("./data/rf_ncv.csv")
ncv_df

In [ ]:
ncv_df["ROC-AUC"].mean()

# Random Forest - Scoring on Test Set

In [ ]:
# Define the grid search object
gscv = GridSearchCV(
    estimator=rf_clf,
    param_grid=param_grid,
    cv=3,
    n_jobs=-1,
    verbose=1,
    scoring="roc_auc", 
)

# Search
gscv_result = gscv.fit(X_train, y_train)

In [ ]:
model = gscv_result.best_estimator_
print("Best Params:")
print(gscv_result.best_params_)
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, y_pred, normalize='true')
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()